In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

from typing import List, Dict

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# Parsing Input Dataset
songs = pd.read_csv('/content/songdata.csv') 

In [6]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [7]:
# Resampling only 5000 random songs for the assignment since the dataset is very huge
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
# Filtering data - removing unwanted "\n" characters
songs['text'] = songs['text'].str.replace(r'\n', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [9]:
# After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word.
# Here, we pay particular attention to the arguments we can specify.
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')


In [11]:
# Creating Lyrics Matrix
# We will use this matrix for recommendation by using Cosine Similarity
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [12]:
# Calculating cosine similarities in the matrix
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [13]:
# Once we get the similarities, we'll store in a dictionary the names of the 50 most similar songs for each song in our dataset.
similarities = {}
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

# Now we can use the similarity scores to access the most similar items and give a recommendation.

In [14]:
# Defining content based recommender class
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [15]:
# Instantiating the class
recommedations = ContentBasedRecommender(similarities)

In [16]:
# Picking a song by index from dataset and making recommendation based on it

recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

recommedations.recommend(recommendation)

The 4 recommended songs for R.u. Satisfied are:
Number 1:
I Ain't Ever Satisfied by John Mellencamp with 0.692 similarity score
--------------------
Number 2:
I'm Satisfied With You by Hank Williams with 0.68 similarity score
--------------------
Number 3:
The Satisfied Mind by Devo with 0.673 similarity score
--------------------
Number 4:
Love Of My Life by Gino Vannelli with 0.249 similarity score
--------------------


In [18]:
# We can redo this as many times for any song from the dataset
recommendation2 = {
    "song": songs['song'].iloc[420],
    "number_songs": 4 
}

recommedations.recommend(recommendation2)

The 4 recommended songs for Suck My Kiss are:
Number 1:
She's Not My Girlfriend by Marilyn Manson with 0.393 similarity score
--------------------
Number 2:
Kiss Kiss by Yeah Yeah Yeahs with 0.287 similarity score
--------------------
Number 3:
Kiss Kiss by Chris Brown with 0.269 similarity score
--------------------
Number 4:
Kiss Me by Olly Murs with 0.24 similarity score
--------------------
